# LSH Algorithm Improvement By Applying Bitmap Indexing

In [7]:
! pip3 install progressbar2
import argparse
import sys
from os import listdir
from os.path import isfile, join
from typing import Dict, List, Optional, Tuple
import imagehash
from PIL import Image
import os, os.path
import cv2
from collections import Counter
import scipy as sp
import numpy as np # Import numpy library 
from skimage.feature import hog # Import Hog model to extract features
from sklearn.metrics import confusion_matrix # Import confusion matrix to evaluate the performance

In [2]:
imgs = []
y = []
file_size = []
k = 0
path = "./data/101_ObjectCategories" # Give the dataset path here

##  Data Preprocessing:
1. Load the images using cv2
2. Image resize
3. Feature extraction: BGR to Gray conversion 
4. Feature extraction: Histogram of Oriented Gradients(HOG)

In [3]:
folder = os.listdir(path) # from the given path get the file names such as accordion, airplanes etc..
for file in folder: # for every file name in the given path go inseide that directory and get the images
    subpath = os.path.join(path,file)  # Join the name of these files to the previous path 
    
    files = os.listdir(subpath) # Take these image names to a list called files
    j = 0
    for i in range(np.size(files)): # now we shall loop through these number of files
        
        im = cv2.imread(subpath+'/'+files[0+j]) # Read the images from this subpath
        
        imgs.append(im) # append all the read images to a list called imgs
        y.append(k) # generate a labe to every file and append it to labels list

        j += 1
        if (j == (np.size(files))):
            file_size.append(j)
   
    k += 1
     
y = np.array(y).tolist()
ix = []
for index, item in enumerate(imgs):
    if (np.size(item) == 1):
        ix.append(index)
        del imgs[index]
        
for index, item in enumerate(y):
    for v in range(np.size(ix)):
        if (index == ix[v]):
            del y[index]
        
y = np.array(y).astype(np.float64) 

# Function to convert an image from color to grayscale
def rgb2gray(rgb):
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    return gray

def resize_(image):
    u = cv2.resize(image,(256,256))
    return u

def fd_hog(image):
    fd = hog(image, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1))
    return fd

In [4]:
a=[]
import progressbar
with progressbar.ProgressBar(max_value=len(imgs)) as bar:
    i=1
    for img in imgs:
        b=resize_(img)
        c=rgb2gray(b)   
        d=fd_hog(c)
        a.append(d)
        bar.update(i)
        i+=1

/Users/fudonghuang/anaconda3/lib/python3.7/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
100% (9176 of 9176) |####################| Elapsed Time: 0:02:10 Time:  0:02:10


## Split data
Split the data to training and validation data. We choose 70% for training and 30% for validation purposes.

In [5]:
# append 'label' and 'id' to the last two colunms
import pandas as pd
df = pd.DataFrame(a)
df['lable'] = y
id_ = np.arange(1,len(df)+1,1)
df['id'] = id_
X = df.values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Using PySpark to retrieve similar images

In [8]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder \
     .master("local") \
     .appName("Image Retrieval") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [10]:
Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])

In [11]:
Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

In [12]:
Train_df.show()

+----+-----+--------------------+
|  id|label|            features|
+----+-----+--------------------+
|7837|   92|[0.10652358713154...|
| 154|    2|[0.0,0.0,0.0,0.0,...|
|2536|   30|[0.22479879532381...|
|2386|   27|[0.23551754302060...|
|3460|   46|[0.21107449806615...|
|9147|  101|[0.44669090633819...|
|6470|   88|[0.0,0.0,0.0,0.0,...|
|1949|   21|[0.09451663745906...|
|5203|   69|[0.16024053531569...|
|1527|   18|[0.93496391565440...|
|1009|   17|[0.03582796048509...|
|2714|   33|[0.33319117176671...|
|6198|   84|[0.14287676217549...|
|7864|   92|[0.20966244965453...|
|5992|   81|[0.0,0.0,0.0,0.0,...|
|6691|   88|[0.0,0.0,0.0,0.0,...|
|4465|   61|[0.29334435596757...|
|4495|   61|[0.29257185269748...|
|4855|   64|[0.28005566153066...|
|6208|   84|[0.03645562561073...|
+----+-----+--------------------+
only showing top 20 rows



In [57]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=7,numHashTables=101)
model = brp.fit(Train_df)

In [58]:
%%time
print("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(Train_df).show()

The hashed dataset where hashed values are stored in the column 'hashes':
+----+-----+--------------------+--------------------+
|  id|label|            features|              hashes|
+----+-----+--------------------+--------------------+
|7837|   92|[0.10652358713154...|[[0.0], [0.0], [-...|
| 154|    2|[0.0,0.0,0.0,0.0,...|[[-1.0], [0.0], [...|
|2536|   30|[0.22479879532381...|[[0.0], [0.0], [-...|
|2386|   27|[0.23551754302060...|[[0.0], [0.0], [0...|
|3460|   46|[0.21107449806615...|[[0.0], [0.0], [0...|
|9147|  101|[0.44669090633819...|[[-1.0], [0.0], [...|
|6470|   88|[0.0,0.0,0.0,0.0,...|[[-1.0], [0.0], [...|
|1949|   21|[0.09451663745906...|[[-1.0], [0.0], [...|
|5203|   69|[0.16024053531569...|[[0.0], [-1.0], [...|
|1527|   18|[0.93496391565440...|[[-1.0], [0.0], [...|
|1009|   17|[0.03582796048509...|[[-1.0], [0.0], [...|
|2714|   33|[0.33319117176671...|[[-1.0], [0.0], [...|
|6198|   84|[0.14287676217549...|[[-1.0], [0.0], [...|
|7864|   92|[0.20966244965453...|[[0.0], [0.0]

In [59]:
%%time
print("Approximately joining Train_df and Test_df on Euclidean distance smaller than 1:")
model.approxSimilarityJoin(Train_df, Test_df, 2, distCol="EuclideanDistance")\
    .select(col("datasetA.id").alias("Train_df"),
            col("datasetB.id").alias("Test_df"),
            col("EuclideanDistance")).show(30)

Approximately joining Train_df and Test_df on Euclidean distance smaller than 1:
+--------+-------+-------------------+
|Train_df|Test_df|  EuclideanDistance|
+--------+-------+-------------------+
|    5160|   5208|                0.0|
|    2420|   2421|                0.0|
|    5867|   5856|                0.0|
|    6424|   7143| 1.9333261469222367|
|    3360|   3358|                0.0|
|    2372|   2377|                0.0|
|    5193|   5196|                0.0|
|    7126|   7093| 0.6086425684921115|
|    3719|   3414| 1.9948791568694886|
|    5188|   5191|                0.0|
|    3724|   3698|                0.0|
|    6578|   6486| 1.8685952653951472|
|    3725|   3675| 1.7261715771631647|
|    5028|   5007|                0.0|
|    8762|   8327|0.08259223937703618|
|      26|      5| 1.3620157872604062|
|    3647|   3614| 1.3185869974384783|
|     420|    429|                0.0|
|       6|     22| 1.1096283367520818|
|    6225|   6229|  1.750789784513495|
|    9023|   9019|    

In [60]:
key = Vectors.dense(X_test[1][0:-2])

In [61]:
X_test[1][-2]

56.0

In [62]:
key

DenseVector([0.3396, 0.0703, 0.0957, 0.256, 0.228, 0.0, 0.0011, 0.0094, 0.0046, 0.0023, 0.0059, 0.3703, 0.6105, 0.003, 0.0027, 0.0008, 0.0103, 0.0039, 0.0138, 0.449, 0.451, 0.0234, 0.026, 0.0226, 0.0165, 0.0035, 0.0156, 0.3277, 0.5871, 0.0233, 0.0206, 0.0057, 0.0704, 0.0128, 0.0221, 0.2452, 0.4467, 0.1044, 0.0524, 0.046, 0.081, 0.0333, 0.0565, 0.2504, 0.2502, 0.2143, 0.0779, 0.0365, 0.1426, 0.1604, 0.1946, 0.177, 0.1847, 0.0394, 0.0539, 0.0475, 0.0708, 0.0249, 0.0632, 0.2163, 0.4787, 0.0248, 0.0352, 0.0861, 0.0132, 0.0075, 0.0167, 0.3854, 0.5094, 0.0206, 0.0167, 0.0304, 0.034, 0.0159, 0.0318, 0.212, 0.6471, 0.0081, 0.0073, 0.0438, 0.0692, 0.0124, 0.01, 0.154, 0.4581, 0.1326, 0.0944, 0.0694, 0.0914, 0.0607, 0.1218, 0.2039, 0.2645, 0.093, 0.058, 0.1068, 0.1202, 0.0779, 0.0956, 0.169, 0.2737, 0.0887, 0.059, 0.116, 0.0776, 0.1082, 0.1067, 0.21, 0.2792, 0.0548, 0.0409, 0.1226, 0.0342, 0.0372, 0.072, 0.2422, 0.3571, 0.0991, 0.0984, 0.0598, 0.1855, 0.028, 0.0281, 0.1287, 0.2517, 0.0646, 0.087

In [63]:
print("Approximately searching Train_df for 2 nearest neighbors of the key:")
result = model.approxNearestNeighbors(Train_df, key, 20)

Approximately searching Train_df for 2 nearest neighbors of the key:


In [64]:
result_id = result.select('id',).collect()
result_id[0].id

4550

In [65]:
result.show()

+----+-----+--------------------+--------------------+------------------+
|  id|label|            features|              hashes|           distCol|
+----+-----+--------------------+--------------------+------------------+
|4550|   61|[0.33334101495930...|[[-1.0], [0.0], [...|3.4806043801545896|
|7501|   91|[0.20710385522880...|[[-1.0], [0.0], [...|3.4834106551402226|
|4537|   61|[0.26592640980482...|[[0.0], [0.0], [-...| 3.507966004541211|
|2475|   29|[0.13772105316623...|[[0.0], [0.0], [0...|3.5197645182540103|
|4987|   65|[0.23697329706829...|[[0.0], [0.0], [0...| 3.522028762887504|
|4464|   61|[0.31955989396811...|[[-1.0], [0.0], [...|  3.52828572030332|
|4928|   65|[0.15990193535334...|[[-1.0], [0.0], [...|3.5329413642061915|
|4495|   61|[0.29257185269748...|[[0.0], [0.0], [0...|3.5423509614570334|
|4446|   61|[0.33936369752198...|[[-1.0], [0.0], [...|3.5475574281624955|
|  95|    1|[0.10591827046929...|[[-1.0], [0.0], [...|  3.58046054545764|
|4523|   61|[0.26169414820258...|[[-1.

In [66]:
list = []
accuracy = 0
numOfNeighbor = 5
with progressbar.ProgressBar(max_value=100) as bar:
    for i in range(0, 100):
        Catg = y_test[i]
        key = Vectors.dense(X_test[i][0:-2])
        result = model.approxNearestNeighbors(Train_df, key, numOfNeighbor)
        temp = Counter([int(row['label']) for row in result.collect()])
        if  Catg in temp:
            curAccuracy = temp.get(Catg)/ numOfNeighbor
            list.append(curAccuracy)
            accuracy += curAccuracy
        else:
            list.append(0)
        bar.update(i)

100% (100 of 100) |######################| Elapsed Time: 0:12:30 Time:  0:12:30


In [67]:
accuracy / 100

0.3619999999999999

In [68]:
list

[1.0,
 0,
 1.0,
 0.4,
 0.2,
 0.8,
 0,
 0.2,
 1.0,
 0.8,
 0.4,
 1.0,
 0.2,
 0,
 1.0,
 0.2,
 0.4,
 0.8,
 0,
 0,
 0,
 0,
 0.2,
 0,
 1.0,
 1.0,
 0,
 0,
 1.0,
 0,
 0,
 0.6,
 0,
 0,
 0.6,
 0.2,
 0,
 1.0,
 1.0,
 1.0,
 0,
 0,
 0,
 0.2,
 0,
 0,
 0,
 0,
 0.6,
 0.8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 0,
 0.4,
 0.4,
 1.0,
 1.0,
 1.0,
 0.2,
 1.0,
 1.0,
 0,
 0,
 0.4,
 1.0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 0,
 0,
 0,
 0.6,
 1.0,
 0.8,
 0.2,
 0,
 0.4,
 0.4,
 0,
 0.4,
 0.6,
 1.0,
 0,
 0.4,
 1.0,
 0.4,
 1.0]

In [ ]:
# from matplotlib.pyplot import imshow
# imshow(imgs[4795])